# **一个Actor-Critic算法的（批处理方式）简单实现**
## **算法概述**
- 采用批输入数据的方式加速训练;
- 批输入的数据包括TD误差以及log概率;
- 采用TD误差进行批处理训练过程及其不稳定，很难训起来。

In [13]:
import copy
import platform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [14]:
if platform.system() == "Darwin":
    PYTORCH_ENABLE_MPS_FALLBACK=1
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
value_method = "TD_error"
all_value_method = ["Q_value","V_value","TD_error","Advantage"]
assert value_method in all_value_method, "You choose a wrong value_method!"

In [15]:
class ActorCriticTrainer(nn.Module):
    def __init__(self, env):
        super(ActorCriticTrainer, self).__init__()
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.create_training_network()
        self.create_training_method()
        self.GAMMA = 0.9
        self.to(device)

        self.state_batch = []
        self.action_batch = []
        self.reward_batch = []
        self.next_state_batch = []

    def create_training_network(self):
        self.fc = nn.Linear(self.state_dim, 20)
        self.critic = nn.Sequential(self.fc,nn.ReLU(),nn.Linear(20,1))
        self.actor = nn.Sequential(self.fc,nn.ReLU(),nn.Linear(20, self.action_dim))

    def create_training_method(self):
        self.optim = optim.Adam(self.parameters(),lr=0.001)
        # self.value_loss = nn.MSELoss()
        # self.actor_loss = nn.LogSoftmax(dim=-1)
    
    def choose_action(self, state):
        with torch.no_grad():
            state = torch.tensor(state, device=device)
            action_probs = F.softmax(self.actor(state), dim=-1)
            action = torch.multinomial(action_probs, 1).item()
            return action

    def calculate_batch_td_error(self, state_batch:torch.Tensor, reward_batch:torch.Tensor):
        value_batch = self.critic(state_batch)
        values = copy.copy(value_batch).squeeze(-1)
        next_values = torch.cat((copy.copy(value_batch[1:,:]).squeeze(-1), torch.tensor([0.], device=device)))
        td_errors = reward_batch + self.GAMMA * next_values - values
        return td_errors

    def calculate_policy_loss(self, state_batch, action_batch, td_errors):
        action_logits_batch = self.actor(state_batch)
        log_probs = torch.log(F.softmax(action_logits_batch, dim=-1))
        action_log_probs = torch.gather(log_probs,1,action_batch.unsqueeze(-1)).squeeze(-1)
        policy_loss = action_log_probs * torch.abs(td_errors)
        return policy_loss

    def perceive(self, state, action, reward, next_state):
        self.state_batch.append(state)
        self.action_batch.append(action)
        self.reward_batch.append(reward)
        self.next_state_batch.append(next_state)

    def train_loop(self):
        state_batch = torch.tensor(self.state_batch, device=device)
        action_batch = torch.tensor(self.action_batch, device=device)
        reward_batch = torch.tensor(self.reward_batch, device=device)

        td_errors = self.calculate_batch_td_error(state_batch, reward_batch)
        value_loss = torch.square(td_errors).mean()
        policy_loss = self.calculate_policy_loss(state_batch, action_batch, td_errors.detach()).mean()
        loss = value_loss - policy_loss
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()
        
    def clear_list(self):
        self.state_batch.clear()
        self.action_batch.clear()
        self.reward_batch.clear()
        self.next_state_batch.clear()

In [16]:
import gym
env_name = "CartPole-v1"
env = gym.make(env_name)
agent = ActorCriticTrainer(env)

In [17]:
import time
def main():
    start_time = time.time()
    for episode in range(3000):
        state, _ = env.reset()
        for step in range(300):
            action = agent.choose_action(state)
            next_state, reward, done, _, _ = env.step(action)
            reward = -1 if done else 0.01
            agent.perceive(state,action,reward,next_state)
            state = next_state
            if done:
                agent.train_loop()
                agent.clear_list()
                break
        if episode % 100 == 0 and episode != 0:
            total_reward = 0
            for i in range(10):
                state, _ = env.reset()
                for step in range(300):
                    action = agent.choose_action(state)
                    next_state, reward, done, _, _ = env.step(action)
                    total_reward += reward
                    state = next_state
                    if done:
                        break
            print(f"episode {episode} total reward is {total_reward/10}")
    end_time = time.time()
    print(f"total time is {end_time - start_time}")

In [18]:
if __name__ == "__main__":
    main()

episode 100 total reward is 20.5
episode 200 total reward is 19.3
episode 300 total reward is 22.2
episode 400 total reward is 28.2
episode 500 total reward is 17.3
episode 600 total reward is 21.7
episode 700 total reward is 29.0
episode 800 total reward is 25.2
episode 900 total reward is 26.1
episode 1000 total reward is 21.1
episode 1100 total reward is 37.1
episode 1200 total reward is 36.1
episode 1300 total reward is 31.0
episode 1400 total reward is 27.0
episode 1500 total reward is 39.0
episode 1600 total reward is 36.1
episode 1700 total reward is 34.4
episode 1800 total reward is 28.9
episode 1900 total reward is 30.1
episode 2000 total reward is 40.5
episode 2100 total reward is 43.1
episode 2200 total reward is 42.4
episode 2300 total reward is 34.4
episode 2400 total reward is 26.8
episode 2500 total reward is 24.6
episode 2600 total reward is 26.0
episode 2700 total reward is 28.4
episode 2800 total reward is 27.7
episode 2900 total reward is 20.4
total time is 381.36419

## 实验记录
1、将td_errors取了绝对值。
结果：效果好了一些，因为td_errors有正有负，如果直接乘log_probs再取绝对值的话loss值会抵消，造成训练缓慢的问题。